In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import geoviews as gv
import holoviews as hv
import hvplot.pandas
import cartopy.crs as ccrs

from shapely.geometry import LineString, Point
from scipy.interpolate import splev, splrep

gv.extension('bokeh')
hv.extension('bokeh')

# 端点を結ぶオブジェクトからメッシュの初期配置を設定

横断方向に15分割

In [ ]:
Jline = gpd.read_file('JLines.geojson', encoding='UTF-8')

In [ ]:
listL = Jline.geometry.values

ny = 15
x = np.zeros((len(listL), ny+1))
y = np.zeros_like(x)

for i, l  in enumerate(listL):
    dy = l.length/ny
    for j in range(ny+1):
        pp = l.interpolate(j*dy) 
        x[i,j] = pp.x
        y[i,j] = pp.y

In [ ]:
parr = []
Iarr = []
Jarr = []
for i in range(x.shape[0]):
    for j in range(x.shape[1]):
        parr.append( Point(x[i,j],y[i,j]) )
        Iarr.append(i)
        Jarr.append(j)

gdfnew = gpd.GeoDataFrame({'geometry': parr , 'I': Iarr, 'J': Jarr })
gdfnew.crs = Jline.crs
out = gdfnew.to_file("MeshPointInitial.geojson", driver='GeoJSON')
del out

# ラプラス方程式

In [ ]:
def relaxation(xi,yi,omega=1.9, nit=300):
    dxi, deta = float(1), float(1)
    x = xi.copy()
    y = yi.copy()
    outerr = []
    for it in range(nit):
        xn = x.copy()
        yn = y.copy()
        err = []
        for i in range(1, x.shape[0]-1):
            for j in range(1, x.shape[1]-1):
                alfa = ((x[i,j+1]-x[i,j-1])/2/deta)**2 + ((y[i,j+1]-y[i,j-1])/2/deta)**2
                beta = (x[i+1,j]-x[i-1,j])/2/dxi*(x[i,j+1]-x[i,j-1])/2/deta + (y[i+1,j]-y[i-1,j])/2./dxi*(y[i,j+1]-y[i,j-1])/2/deta
                gamma = ((x[i+1,j]-x[i-1,j])/2/dxi)**2 + ((y[i+1,j]-y[i-1,j])/2/dxi)**2.
                
                # SOR
                dd = (alfa*(x[i+1,j]+x[i-1,j])/dxi**2 \
                          - 2*beta*(x[i+1,j+1]-x[i-1,j+1]-(x[i+1,j-1]-x[i-1,j-1]))/4/dxi/deta \
                          + gamma*(x[i,j+1]+x[i,j-1])/deta**2)/(2*alfa/dxi**2+2*gamma/deta**2) 
                
                err.append( dd - x[i,j] )
                x[i,j] += omega*(dd - x[i,j])
                
                dd = (alfa*(y[i+1,j]+y[i-1,j])/dxi**2 \
                          - 2*beta*(y[i+1,j+1]-y[i-1,j+1]-(y[i+1,j-1]-y[i-1,j-1]))/4/dxi/deta \
                          + gamma*(y[i,j+1]+y[i,j-1])/deta**2)/(2*alfa/dxi**2+2*gamma/deta**2)
                
                err.append( dd - y[i,j] )
                y[i,j] += omega*(dd - y[i,j])
        
        serr = np.sqrt( np.sum(np.array(err)**2) )
        print(serr)
        outerr.append(serr)
        
    return x,y,outerr

#     if (it == 1) or (it == 5) or (it == 10):
#         parr = []
#         Iarr = []
#         Jarr = []
#         for i in range(x.shape[0]):
#             for j in range(x.shape[1]):
#                 parr.append( Point(x[i,j],y[i,j]) )
#                 Iarr.append(i)
#                 Jarr.append(j)
                
#         gdfnew = gpd.GeoDataFrame({'geometry': parr , 'I': Iarr, 'J': Jarr })
#         gdfnew.crs = Jline.crs
#         out = gdfnew.to_file("MeshPoint" + str(it).zfill(4) + ".geojson", driver='GeoJSON')
#         del out

In [ ]:
xn, yn, err = relaxation(x,y,omega=1.7)

In [ ]:
gdfout = gpd.GeoDataFrame({'geometry': [Point(xn[i,j],yn[i,j]) for i in range(xn.shape[0]) for j in range(xn.shape[1])]
                           , 'I': [i for i in range(xn.shape[0]) for j in range(xn.shape[1])]
                           , 'J': [j for i in range(xn.shape[0]) for j in range(xn.shape[1])]
                          }, crs=Jline.crs)

out = gdfout.to_file("MeshPointOpt.geojson", driver='GeoJSON')

#  check

In [ ]:
g0 = gdfnew.hvplot(label='initial')
g1 = gdfout.hvplot(label='optimized')
back = gv.WMTS('https://mt1.google.com/vt/lyrs=s&x={X}&y={Y}&z={Z}', name="GoogleMapsImagery")
gall = g0*g1*back
gallo = gall.options(width=600,height=500)
gallo

# test:relaxation method

In [ ]:
xn, yn, err = relaxation(x,y,omega=1.7)
xn2, yn2, err2 = relaxation(x,y,omega=1.0)

In [ ]:
g = hv.Curve(err, label='SOR:omega=1.7') * hv.Curve(err2, label='Gauss-Seidel')
go = g.options(logy=True,xlabel='iteration', ylabel='error',width=400, height=350, show_grid=True)
go

In [ ]:
hvplot.save(go,'compareRelax.html')